In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import string
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding,Dense
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_tweet= pd.read_csv('data/train.csv')
test_tweet=pd.read_csv('data/test.csv')

In [3]:
df_train = train_tweet.copy()
df_test = test_tweet.copy()

In [4]:
#remove HTML
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [5]:
#remove http
def remove_http(text):
    http=re.compile(r'http\S+')
    return http.sub(r'',text)

In [6]:
#remove URL
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [7]:
#remove punctuation
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [8]:
#tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text

In [9]:
#method to put all of the above together
def clean_text(text):
    text=remove_html(text)
    text=remove_http(text)
    text=remove_URL(text)
    text=remove_punct(text)
    text=tokenize(text)
    return text

In [10]:
def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#delete stopwords
df_train['text']=df_train['text'].apply(lambda x : delete_stopwords(x))

In [15]:
#apply clean_text method to df
df_train['text']=df_train['text'].apply(lambda x : clean_text(x))

In [16]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,"[Deeds, Reason, earthquake, May, ALLAH, Forgiv...",1
1,4,NaN,NaN,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,5,NaN,NaN,"[residents, asked, shelter, place, notified, o...",1
3,6,NaN,NaN,"[13000, people, receive, wildfires, evacuation...",1
4,7,NaN,NaN,"[got, sent, photo, Ruby, Alaska, smoke, wildfi...",1


In [20]:
#tokenize
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(df_train['text'])
word_index_dict=tokenizer_obj.word_index
#num of unique words
num_words=len(word_index_dict)+1

In [21]:
print(num_words)

18076


In [22]:
#use GloVe for word embedding
#dictionary with the words as keys and the embeddings as values
def glove_embedding(df):
    embeddings_index = {}
    f = open('glove/glove.6B.100d.txt',encoding='utf8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:])
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [24]:
embeddings_dict = glove_embedding(df_train)

Found 400000 word vectors.


In [25]:
#create a method to create a matrix with the embeddings
def create_embedding_matrix(df):
    embedding_matrix=np.zeros((num_words,100))
    for word,i in word_index_dict.items():
        if i > num_words:
            continue
        emb_vec=embeddings_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i]=emb_vec
    return embedding_matrix

In [26]:
embedding_matrix=create_embedding_matrix(df_train)

In [73]:
def create_cnn_model(embedding_matrix, input_length):
    model = Sequential()
    model.add(Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                        weights=[embedding_matrix], input_length=input_length, trainable=False))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_cnn_model(embedding_matrix, 50)

In [70]:
# def create_model(embedding_matrix, input_length):
#     model =tf.keras.models.Sequential([
#         tf.keras.layers.Embedding(num_words,100,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
#                                   input_length=50,trainable=False),
#         tf.keras.layers.Conv1D(128,7,activation='relu'),
#         tf.keras.layers.Conv1D(256, 7, activation='relu'),
#         tf.keras.layers.Conv1D(128, 7, activation='relu'),
#         # LSTM(64, dropout=0.2, recurrent_dropout=0.2),
#         tf.keras.layers.GlobalMaxPooling1D(),
#         tf.keras.layers.Dense(64,activation='relu'),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(1,activation='sigmoid')
#     ])
#
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     return model
#
# model = create_model(embedding_matrix, 50)

In [74]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 50, 100)           1807600   
                                                                 
 conv1d_30 (Conv1D)          (None, 46, 128)           64128     
                                                                 
 global_max_pooling1d_14 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_28 (Dense)            (None, 64)                8256      
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_29 (Dense)            (None, 1)                 65        
                                                     

In [265]:
sequences = tokenizer_obj.texts_to_sequences(df_train['text'])
MAX_LEN = 50
tweet_pad = pad_sequences(sequences, maxlen=MAX_LEN, truncating='post', padding='post')

In [248]:
train=tweet_pad[:df_train.shape[0]]
test=tweet_pad[:df_train.shape[0]]

In [318]:
#split data into train and test
X_train,X_test,y_train,y_test=train_test_split(train,df_train['target'].values,test_size=0.2)

In [319]:
#fit model
history=model.fit(X_train,y_train,batch_size=4,epochs=10,validation_data=(X_test,y_test),verbose=2)
# model.fit(X_train,y_train,batch_size=4,epochs=10,validation_data=(X_test,y_test),verbose=2)

Epoch 1/10
1523/1523 - 3s - loss: 0.0569 - accuracy: 0.9785 - val_loss: 0.0366 - val_accuracy: 0.9842 - 3s/epoch - 2ms/step
Epoch 2/10
1523/1523 - 3s - loss: 0.0387 - accuracy: 0.9806 - val_loss: 0.0383 - val_accuracy: 0.9810 - 3s/epoch - 2ms/step
Epoch 3/10
1523/1523 - 3s - loss: 0.0377 - accuracy: 0.9811 - val_loss: 0.0427 - val_accuracy: 0.9823 - 3s/epoch - 2ms/step
Epoch 4/10
1523/1523 - 3s - loss: 0.0426 - accuracy: 0.9796 - val_loss: 0.0499 - val_accuracy: 0.9803 - 3s/epoch - 2ms/step
Epoch 5/10
1523/1523 - 3s - loss: 0.0415 - accuracy: 0.9798 - val_loss: 0.0676 - val_accuracy: 0.9816 - 3s/epoch - 2ms/step
Epoch 6/10
1523/1523 - 3s - loss: 0.0370 - accuracy: 0.9821 - val_loss: 0.0534 - val_accuracy: 0.9829 - 3s/epoch - 2ms/step
Epoch 7/10
1523/1523 - 3s - loss: 0.0433 - accuracy: 0.9803 - val_loss: 0.0624 - val_accuracy: 0.9796 - 3s/epoch - 2ms/step
Epoch 8/10
1523/1523 - 3s - loss: 0.0449 - accuracy: 0.9801 - val_loss: 0.0788 - val_accuracy: 0.9790 - 3s/epoch - 2ms/step
Epoch 9/

In [325]:
sample_submission = pd.read_csv("data/sample_submission.csv")
df_test
df_test.shape[0]

3263

In [329]:
sequences = tokenizer_obj.texts_to_sequences(df_test['text'])
MAX_LEN = 50
tweet_pad = pad_sequences(sequences, maxlen=MAX_LEN, truncating='post', padding='post')

In [330]:
test=tweet_pad[:df_test.shape[0]]
print(test)

[[ 2909   772   240 ...     0     0     0]
 [  348 13456   154 ...     0     0     0]
 [  974  1032   240 ...     0     0     0]
 ...
 [  831   600   243 ...     0     0     0]
 [ 6129   472   364 ...     0     0     0]
 [ 5811  6357  2648 ...     0     0     0]]


In [377]:
sample_submission["target"] = model.predict(test)
sample_submission["target"] = sample_submission["target"].apply(lambda x: 1 if x > 0.3 else 0)

102/102 [==============================] - 0s 2ms/step


In [380]:
# sample_submission.head()
sample_submission.to_csv("submission.csv", index=False)
sample_submission['target'].value_counts()

target
0    2356
1     907
Name: count, dtype: int64

In [379]:
ms = pd.read_csv("data/ms.csv")
submission = pd.read_csv("submission.csv")

# Compare target columns
comparison = ms['target'] == submission['target']

# Create a new DataFrame with the comparison results
comparison_df = pd.DataFrame({'ms_target': ms['target'], 'submission_target': submission['target'], 'correct': comparison})

# Display the comparison DataFrame
# print(comparison_df)
comparison_df['correct'].value_counts()

correct
True     2597
False     666
Name: count, dtype: int64

In [ ]:
# 0.3
True,2597
False,666




